In [1]:
import os
import sys
import random
from pathlib import Path
import osgeo  #Needed for use on Windows only
import matplotlib.pyplot as plt
from rasterio import plot
from rasterio.plot import show
import geopandas as gpd
from datetime import datetime
import sqlalchemy as sa
import pandas as pd
%matplotlib inline

In [2]:
sys.path.append(r"../../src/collectcube")
from samplegen import *
from db import *

In [3]:
out_dir = 'C:/GISprojects/ParaguayTraining'
local_db_path = os.path.join(out_dir, 'py_training.db')
## reference .tif with target crs and resolution
ref_ras = Path("../../data/CEL_py/samp_ras.tif")

res = 10

## Create new pixel table from scratch

In [ ]:
 # see create_db notebook

## Format existing points

### Get max pid in current table (to assign new ids to points to add)

In [107]:
maxpid = get_max_id_in_db(local_db_path)
print(f'current max pid is {maxpid}')

current max pid is 31471


Need to make sure that the new pixel data has all of the same fields as the existing table. Need to pre-fill PID0 (starting with current max from above + 1), PID1 (0 usually for new points), PID (concat(PID,"_"PID1), cent_X & cent_Y (in project crs - EPSG 8858 here), and sampgroup. Save as CSV file to load below.

In [109]:
old_pts = pd.read_csv('C:/Users/klobw/Desktop/last_last.csv')
#old_pts = pd.read_csv(os.path.join(out_dir,'lastpts.csv'))
old_pts_df = gpd.GeoDataFrame(old_pts,geometry = gpd.points_from_xy(old_pts.cent_X, old_pts.cent_Y, crs="EPSG:8858"))
print(old_pts_df[old_pts_df['PID0'] == 31449])

Empty GeoDataFrame
Columns: [PID, Center, cent_X, cent_Y, cent_lat, cent_long, ransamp, checked, PID0, PID1, sampgroup, rand, rand2, geometry]
Index: []


In [110]:
old_pts_df.to_crs('4326')
old_pts_df['cent_long'] = old_pts_df['geometry'].x
old_pts_df['cent_lat'] = old_pts_df['geometry'].y
old_pts_df.to_crs("EPSG:8858")
print(old_pts_df[old_pts_df['PID0'] == 31449])

Empty GeoDataFrame
Columns: [PID, Center, cent_X, cent_Y, cent_lat, cent_long, ransamp, checked, PID0, PID1, sampgroup, rand, rand2, geometry]
Index: []


In [111]:
old_pts_df.head()

,PID,Center,cent_X,cent_Y,cent_lat,cent_long,ransamp,checked,PID0,PID1,sampgroup,rand,rand2,geometry
0,31472_0,1,3101422,-3432654,-3432654.0,3101422.0,0,0,31472,0,GE_extra,0.907977,0.050668,POINT (3101422.000 -3432654.000)
1,31473_0,1,3101663,-3432693,-3432693.0,3101663.0,0,0,31473,0,GE_extra,0.972399,0.391651,POINT (3101663.000 -3432693.000)
2,31474_0,1,3101443,-3432713,-3432713.0,3101443.0,0,0,31474,0,GE_extra,0.356496,0.106146,POINT (3101443.000 -3432713.000)
3,31475_0,1,3080536,-3095766,-3095766.0,3080536.0,0,0,31475,0,GE_extra,0.144717,0.524063,POINT (3080536.000 -3095766.000)
4,31476_0,1,3080917,-3095723,-3095723.0,3080917.0,0,0,31476,0,GE_extra,0.481875,0.146777,POINT (3080917.000 -3095723.000)


In [112]:
ptfile = os.path.join(out_dir,'lastPtsLAST2_cent.shp')
centered_pts = move_points_to_pixel_centroids(old_pts_df, ref_ras, write_pts=True, ptsout=ptfile)

ref_ras has crs:EPSG:8858


In [57]:
print(centered_pts.loc[31449])

geometry    POINT (3070020.000 -3048540.000)
Name: 31449, dtype: geometry


In [113]:
all_pts = get_full_point_file(centered_pts, pt_file_out=None, res=res, lastpt=-1, write_pts=False)
all_pts.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
there are 88 neighbor pixels
there are 99 total pixels


,geometry,PID,Center
31472,POINT (3101430.000 -3432650.000),0031472_0,1
31473,POINT (3101670.000 -3432690.000),0031473_0,1
31474,POINT (3101450.000 -3432710.000),0031474_0,1
31475,POINT (3080550.000 -3095780.000),0031475_0,1
31476,POINT (3080930.000 -3095730.000),0031476_0,1


In [114]:
boxfile = os.path.join(out_dir,'training_boxes_lastPtsLAST2.shp')
make_pixel_boxes_from_pts(all_pts, boxfile, res, write_boxes=True)

,geometry
0,"POLYGON ((3101435.000 -3432655.000, 3101435.00..."
1,"POLYGON ((3101675.000 -3432695.000, 3101675.00..."
2,"POLYGON ((3101455.000 -3432715.000, 3101455.00..."
3,"POLYGON ((3080555.000 -3095785.000, 3080555.00..."
4,"POLYGON ((3080935.000 -3095735.000, 3080935.00..."
...,...
94,"POLYGON ((3081005.000 -3094065.000, 3081005.00..."
95,"POLYGON ((3081025.000 -3094065.000, 3081025.00..."
96,"POLYGON ((3081005.000 -3094075.000, 3081005.00..."
97,"POLYGON ((3081015.000 -3094075.000, 3081015.00..."


In [115]:
pts_info = make_pixel_table(all_pts,samp_group=None)
pts_info.drop(columns=['sampgroup'], inplace=True)
#old_pts.rename({'sampGroup': 'sampgroup'}, axis='columns', inplace=True)  #Fixing mistake
pts_info2 = pd.merge(pts_info,old_pts[['PID0','sampgroup','rand','rand2']],on='PID0', how='inner')
pts_info2.head()

,PID,Center,cent_X,cent_Y,cent_lat,cent_long,ransamp,checked,PID0,PID1,sampgroup,rand,rand2
0,0031472_0,1,3101430.0,-3432650.0,-27.239530,-55.813895,1,0,31472,0,GE_extra,0.907977,0.050668
1,0031472_1,0,3101420.0,-3432640.0,-27.239447,-55.814017,1,0,31472,1,GE_extra,0.907977,0.050668
2,0031472_2,0,3101430.0,-3432640.0,-27.239447,-55.813906,1,0,31472,2,GE_extra,0.907977,0.050668
3,0031472_3,0,3101440.0,-3432640.0,-27.239447,-55.813796,1,0,31472,3,GE_extra,0.907977,0.050668
4,0031472_4,0,3101420.0,-3432650.0,-27.239530,-55.814005,1,0,31472,4,GE_extra,0.907977,0.050668


In [100]:
#print(pts_info2[pts_info2['PID0'] == 31449])

### To append new pixels to existing table (new pixels formatted above) 

In [118]:
make_pixel_table_in_db(vals2, local_db_path, treat_existing='append')
table_check = check_table(local_db_path,'pixels')

              PID  Center     cent_X     cent_Y   cent_lat  cent_long  \
277510  0031482_4       0  3081000.0 -3094060.0 -24.458177 -56.400685   
277511  0031482_5       0  3081020.0 -3094060.0 -24.458177 -56.400467   
277512  0031482_6       0  3081000.0 -3094070.0 -24.458259 -56.400675   
277513  0031482_7       0  3081010.0 -3094070.0 -24.458259 -56.400566   
277514  0031482_8       0  3081020.0 -3094070.0 -24.458259 -56.400457   

        ransamp  checked   PID0  PID1 sampgroup      rand     rand2 biome  \
277510        1        0  31482     4  GE_extra  0.564274  0.539983    BH   
277511        1        0  31482     5  GE_extra  0.564274  0.539983    BH   
277512        1        0  31482     6  GE_extra  0.564274  0.539983    BH   
277513        1        0  31482     7  GE_extra  0.564274  0.539983    BH   
277514        1        0  31482     8  GE_extra  0.564274  0.539983    BH   

        gridCell  
277510      3867  
277511      3867  
277512      3867  
277513      3867  
277

### To update existing records in table

###### Adding new fields
For example here we want to add a field called gridCell containing the value of the 'gridCell' (from a polygon file) each point falls in and a field called 'biome' containing the biome (from a raster file) each point falls in

In [ ]:
con = sqlite3.connect(local_db_path)
c = con.cursor()
## adding a new column to existing table
c.execute("ALTER TABLE pixels ADD COLUMN biome TEXT")
con.commit()
c.execute("ALTER TABLE pixels ADD COLUMN gridCell INTEGER")
con.commit()
c.close()

In [120]:
## Load pixel table
conn = sqlite3.connect(local_db_path, isolation_level=None,
    detect_types=sqlite3.PARSE_COLNAMES)
db_df = pd.read_sql_query("SELECT * FROM pixels", conn)
db_df.tail()

,PID,Center,cent_X,cent_Y,cent_lat,cent_long,ransamp,checked,PID0,PID1,sampgroup,rand,rand2,biome,gridCell
277510,0031482_4,0,3081000.0,-3094060.0,-24.458177,-56.400685,1,0,31482,4,GE_extra,0.564274,0.539983,BH,3867
277511,0031482_5,0,3081020.0,-3094060.0,-24.458177,-56.400467,1,0,31482,5,GE_extra,0.564274,0.539983,BH,3867
277512,0031482_6,0,3081000.0,-3094070.0,-24.458259,-56.400675,1,0,31482,6,GE_extra,0.564274,0.539983,BH,3867
277513,0031482_7,0,3081010.0,-3094070.0,-24.458259,-56.400566,1,0,31482,7,GE_extra,0.564274,0.539983,BH,3867
277514,0031482_8,0,3081020.0,-3094070.0,-24.458259,-56.400457,1,0,31482,8,GE_extra,0.564274,0.539983,BH,3867


In [116]:
## To fill the biome field

def get_val_at_XY(img,df,colname):
    coords = [(x,y) for x, y in zip(df.cent_X, df.cent_Y)]
    with rio.open(img) as src:
        pts = [x[0] for x in src.sample(coords)]
    df[colname] = pts

    return df

forstrat = 'C:/Users/klobw/Desktop/NasaProject/Paraguay/GIS_datasets/forest_strata3.tif'
#vals = get_val_at_XY(forstrat,db_df,'biome')
forstrat_key = {1:'Chaco', 2:'BS',3:'Cer', 4:'BH'}
vals['biome'] = vals['biome'].replace(forstrat_key)
vals.tail()

,PID,Center,cent_X,cent_Y,cent_lat,cent_long,ransamp,checked,PID0,PID1,sampgroup,rand,rand2,biome
94,0031482_4,0,3081000.0,-3094060.0,-24.458177,-56.400685,1,0,31482,4,GE_extra,0.564274,0.539983,BH
95,0031482_5,0,3081020.0,-3094060.0,-24.458177,-56.400467,1,0,31482,5,GE_extra,0.564274,0.539983,BH
96,0031482_6,0,3081000.0,-3094070.0,-24.458259,-56.400675,1,0,31482,6,GE_extra,0.564274,0.539983,BH
97,0031482_7,0,3081010.0,-3094070.0,-24.458259,-56.400566,1,0,31482,7,GE_extra,0.564274,0.539983,BH
98,0031482_8,0,3081020.0,-3094070.0,-24.458259,-56.400457,1,0,31482,8,GE_extra,0.564274,0.539983,BH


In [117]:
## To fill the gridCell field

def get_val_at_XY_poly(poly_file,df,colname):
    polys = gpd.read_file(poly_file)
    points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(x=df.cent_X, y=df.cent_Y), crs=polys.crs)
    sj = gpd.sjoin(left_df=points, right_df=polys, how="left", rsuffix="ply_")
    df[colname]=sj['UNQ'].astype('Int64')
    return df

grids = 'C:/Users/klobw/Desktop/NasaProject/LUCinLA_grid_8858.gpkg'
vals2 = get_val_at_XY_poly(grids,vals,'gridCell')
vals2.tail()

,PID,Center,cent_X,cent_Y,cent_lat,cent_long,ransamp,checked,PID0,PID1,sampgroup,rand,rand2,biome,gridCell
94,0031482_4,0,3081000.0,-3094060.0,-24.458177,-56.400685,1,0,31482,4,GE_extra,0.564274,0.539983,BH,3867
95,0031482_5,0,3081020.0,-3094060.0,-24.458177,-56.400467,1,0,31482,5,GE_extra,0.564274,0.539983,BH,3867
96,0031482_6,0,3081000.0,-3094070.0,-24.458259,-56.400675,1,0,31482,6,GE_extra,0.564274,0.539983,BH,3867
97,0031482_7,0,3081010.0,-3094070.0,-24.458259,-56.400566,1,0,31482,7,GE_extra,0.564274,0.539983,BH,3867
98,0031482_8,0,3081020.0,-3094070.0,-24.458259,-56.400457,1,0,31482,8,GE_extra,0.564274,0.539983,BH,3867


In [69]:
#vals2.to_csv(os.path.join('C:/GISprojects/ParaguayTraining','allPix.csv'), index=False)

###### Updating data in existing fields from Pandas db

In [129]:
## Note this is slow if completely filling a new column 
##   but better than deleting all records because of auto-incremented record number

def update_data(df,update_col,con):
    c = con.cursor()
    s_update = ""
    for i in range(len(df)):
        s_update += "update pixels set biome = '%s' where PID = '%s';"%(df[update_col][i], df['PID'][i])
    c.executescript(s_update)
    con.commit()
    c.close()
    
con = sqlite3.connect(local_db_path)
## NOTE need to repace column name in the method above. TODO fix this
#update_data(vals2,'biome',con)

###### Updating data in existing fields with arguments

In [124]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE pixels SET cent_X = '2772120.0' WHERE PID0 = '30387'")
con.commit()
c.close()
'''
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE pixels SET sampgroup = 'CAN' WHERE sampgroup = 'CANcen+GE_KW'")
con.commit()
c.close()
'''
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE PixelVerification SET imgDate = '2013-07-23' WHERE imgDate ='7/23/2013'")
con.commit()
c.close()
'''

'\ncon = sqlite3.connect(local_db_path)\nc = con.cursor()\nc.execute("UPDATE PixelVerification SET imgDate = \'2013-07-23\' WHERE imgDate =\'7/23/2013\'")\ncon.commit()\nc.close()\n'

###### Deleting records -- WATCH OUT - there is not recovery after commiting

In [ ]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("DELETE FROM pixels WHERE sampgroup = 'Ground_AtaValid'")
con.commit()
c.close()
'''

###### Updating lat long coords after adjusting cent_X or cent_Y
(When viewing a point, it is sometimes clear that the point does not quite fall within the field / land cover it is labeled for (this is especially common in ground data collected from the road). The point can easily be moved by updating cent_X or cent_Y with the methods above if these are in meters/feet (taking care to move in increments matching the pixel resolution so that the new point will be at a pixel center). After such adjustments, the lat/long fields need adjusting as well (this is better done in larger batches at the end of any editing sessions)   

In [101]:
## Load pixel table
#conn = sqlite3.connect(local_db_path, isolation_level=None,
#    detect_types=sqlite3.PARSE_COLNAMES)
#df = pd.read_sql_query("SELECT * FROM pixels", conn)

gdf = gpd.GeoDataFrame(df,geometry = gpd.points_from_xy(df.cent_X, df.cent_Y, crs="EPSG:8858"))
gdfll = gdf.to_crs('4326')
#print(gdfll[gdfll['PID0'] == 30387])
gdfll['cent_long'] = gdfll['geometry'].x
gdfll['cent_lat'] = gdfll['geometry'].y
gdfutm = gdfll.to_crs("EPSG:8858")
gdfutm.drop(columns=['geometry'],inplace=True)
#print(gdfutm[gdfutm['PID0'] == 30387])
#gdfutm.to_csv(os.path.join('D:/NasaProject/temp_backup','allPix_adjusted.csv'), index=False)

# Now use method in "Updating data in existing fields from Pandas db" above to update cent_long and cent_lat fields from gdfutm

## Make other db tables

In [10]:
lc_lut = Path("../../data/Class_LUT.csv")
make_LC_table_from_lut(lc_lut, local_db_path, treat_existing='fail')
make_simp_LC_table(lc_lut, local_db_path, [0,30,35,40,80,98], treat_existing='fail')
make_db(local_db_path)

added LC5 and PixVar tables to db


## checks:

#### inspect a single record

In [ ]:
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("SELECT * FROM PixelVerification WHERE recID = '3573'")
con.commit()
print(c.fetchone()) 
c.close()

#### Make sure table looks right and is correctly formatted

In [ ]:
engine = create_engine('sqlite:///'+ local_db_path)
with engine.connect() as con:
    df = pd.read_sql_table('pixels', con)
df.head()

In [11]:
engine = create_engine('sqlite:///'+ local_db_path)
with engine.connect() as conn:
    lc5_table = Table('LC5', MetaData(), autoload_with=engine)
    rp = conn.execute(lc5_table.select())
results = rp.fetchall() 
print(results)

[(0, ' ------'), (10, 'NoVeg'), (20, 'LowVeg'), (40, 'MedVeg'), (50, 'HighVeg'), (70, 'Trees'), (99, 'Unknown')]


In [99]:
#make_pixel_table_in_db(gdfutm, local_db_path, treat_existing='append')
table_check = check_table(local_db_path,'pixels')

              PID  Center     cent_X     cent_Y   cent_lat  cent_long  \
277411  0031471_4       0  3145070.0 -3123500.0 -24.698683 -55.671875   
277412  0031471_5       0  3145090.0 -3123500.0 -24.698683 -55.671657   
277413  0031471_6       0  3145070.0 -3123510.0 -24.698765 -55.671865   
277414  0031471_7       0  3145080.0 -3123510.0 -24.698765 -55.671756   
277415  0031471_8       0  3145090.0 -3123510.0 -24.698765 -55.671647   

        ransamp  checked   PID0  PID1 sampgroup      rand     rand2 biome  \
277411        0        0  31471     4  GE_extra  0.110972  0.142119    BH   
277412        0        0  31471     5  GE_extra  0.110972  0.142119    BH   
277413        0        0  31471     6  GE_extra  0.110972  0.142119    BH   
277414        0        0  31471     7  GE_extra  0.110972  0.142119    BH   
277415        0        0  31471     8  GE_extra  0.110972  0.142119    BH   

        gridCell  
277411      3972  
277412      3972  
277413      3972  
277414      3972  
277

If the above fails, can convert to pandas df and export to find errors (then update with the methods above to fix)

In [5]:
## This uses only sqllite and can be used to examine table if the above fails
conn = sqlite3.connect(local_db_path, isolation_level=None,
    detect_types=sqlite3.PARSE_COLNAMES)
db_df = pd.read_sql_query("SELECT * FROM PixelVerification", conn)
db_df.tail()
#db_df.to_csv(os.path.join('C:/GISprojects/ParaguayTraining','LatestPixDbInfo.csv'), index=False)

In [130]:
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("SELECT * FROM pixels WHERE PID0 = '12116'")
con.commit()
print(c.fetchone()) 
c.close()

('0012116_0', 1, 3158690.0, -3242170.0, -25.670631889773432, -55.397545216907936, 0, 0, 12116, 0, 'rd_samp', 0.237514246, 0.404479651, 'BH', 3978)
